**Pandas! - DABP14000 - 6 September 2022**

Group Members: 

Przemyslaw Zajac,
Ming Xie,
Erin Criste 


# **Project Summary** 

## Project Goal:

This project is designed to identify the possible factors that affect the full-time data scientists’ salaries in US and predict the salaries based on the employee’s features (experience level, job title, residence, etc.) and the employer’s features (company size, remote ratio, etc.). The project goal is to estimate how much salary a data scientist employee can expect to get as accurately as possible using statistical models based on their professional situation. 

## Project target audience:

This project is meant to benefit the people who are interested in starting their career in the data science field like new grads with data-related majors, and who have been working as a data scientist and looking for career development. They can evaluate their current offer, predict their future income and improve their career plans by using this project. 

The project can also benefit a variety of employers that plan to hire data scientist employees to estimate the hiring costs. 




In [ ]:
# Allows multiple outputs from a single cell:
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
!pip -q install -U statsmodels > log.txt   # ensures no FutureWarnings from statsmodels

import numpy as np, pandas as pd, matplotlib.pyplot as plt, scipy, seaborn as sns, statsmodels.api as sm, pprint
import sklearn, platform, matplotlib, datetime
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

#Point to github repo for datasource
url = 'https://raw.githubusercontent.com/erinlc27/DABP14000_Fall2022/main/ds_salaries.csv'

df = pd.read_csv(url)

In [ ]:
#Print list of columns
print(list(df.columns))

In [ ]:
#Drop duplicate index imported
df.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
#Display 1st 10 observations
display(df.iloc[:10])

In [ ]:
#Look at column types
print(display(df.info()))

#**Exploratory Data Analysis (EDA)**

#1. Dataset Description 

###*Data Scientist Job Salaries: Click here to view/download [dataset](https://www.kaggle.com/datasets/ruchi798/data-science-job-salaries)*

## The Data Scientist Job Salaries dataset was downloaded from Kaggle and contains 607 observations and 12 columns. Each observation provides the employment details of data scientists including experience, salary, and job title to name a few. See below for the column names and description.

```
Column                   Description
-------------------------------------------------------------------------------
work_year                The year the salary was paid.
experience_level         The experience level in the job during the year with the following possible values: EN Entry-level, Junior MI Mid-level, Intermediate SE Senior-level, Expert EX Executive-level, and Director
employment_type          The type of employment for the role: PT Part-time FT, FT Full-time, CT Contract, and FL Freelance
job_title                The role worked in during the year.
salary                   The total gross salary amount paid.
salary_currency          The currency of the salary paid as an ISO 4217 currency code.
salaryinusd              The salary in USD (FX rate divided by avg. USD rate for the respective year via fxdata.foorilla.com).
employee_residence       Employee's primary country of residence in during the work year as an ISO 3166 country code.
remote_ratio             The overall amount of work done remotely, possible values are as follows: 0 No remote work (less than 20%), 50 Partially remote, and 100 Fully remote (more than 80%)
company_location         The country of the employer's main office or contracting branch as an ISO 3166 country code.
company_siz              The average number of people that worked for the company during the year: S less than 50 employees (small), M 50 to 250 employees (medium), and L more than 250 employees (large)
```

The dependent variable used for this analysis is the **SALARY in USD**.

In [ ]:
#Filter dataset to Full Time Employment & company location of US
#Full time employment covers 588 out of 607 observations in the dataset thus we felt it represents most of our observations and filtered out Non-FT observations
#In the initial model we are are only going to explore US companies Data Scientist salary data
df_flt = df.loc[ (df['employment_type'] == 'FT') & (df['company_location'] == 'US') ]
df_flt = df_flt.drop(['salary'], axis=1) #Drop salary column as our dependant variable will be salary in USD

In [ ]:
#Look at high level data stats
dfStat = df_flt.describe()
#Drop count stat
dfStat.drop(index=['count'],inplace=True)
#Drop work_year column, stats for this column are not that informative
dfStat.drop(columns=['work_year'],inplace=True)

dfStat.loc['IQR'] = dfStat.loc['75%'] - dfStat.loc['25%']
dfStat = dfStat.append(df.reindex(dfStat.columns, axis=1).agg(['skew', 'mad', 'kurt']))

mapper = {'salary':'{0:.2f}', 'salary_in_usd':'{0:.2f}', 'remote_ratio':'{0:.0f}'}
print(display(dfStat.style.format(mapper)))

#2. Dataset Features
###To analyze our dataset for our problem statement, we chose to filter to full time employment observations only. Full time employment covers 588 out of 607 observations in the dataset thus we felt it represents most of our observations. We filtered the data to include ONLY United States (US) observations which resulted in 355 records from the overall 607. To reduce the complexity of the initial models we chose to only concentrate on US based employers. We also removed salary feature as our dependant variable will be salary in USD.

###In addition, there were no NULL or missing values in this dataset.

#3. Data Types
###The dataset contains several categorical variables that increase the complexity of the analysis including Experience Level (4 types), Remote Ratio (3 types), Company Size (3 types), and Job Title (40 types). For initial modeling purposes, we will exclude the job titles for easier analysis. The experience level and company size will likely need to be categorized into numeric values for the analysis. 

###From initial exploration of the dataset, we believe that the variables including experience level, salaries in USD, remote ratio, and company size will be the ones to drive our analysis.


In [ ]:
#look at KDE Plot
pair_plot = sns.pairplot(df_flt, diag_kind='kde',plot_kws={"s": 3});
pair_plot.fig.suptitle("Pairwise Distributions");

In [ ]:
#Look at correlation matrix for continious variables
plt.figure(figsize=(12, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
heatmap = sns.heatmap(df_flt.corr(),annot=True, fmt='.2g', cmap=cmap)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);


#4. Correlation
Correlation matrix, histograms, scatterplots, and heatmaps were used to further explore the filtered dataset to begin our initial modeling and analysis.

###Histograms and Scatterplots
The Histograms show skew within the dataset. Work year and remote ratio both are multi-modal plots. This makes sense as these features are categorical variables represented by numerics.

Salary_in_USD has right skew with long tail. 

From the scatterplots, there does not appear to be any linear relationships within the data.

###Correlation Matrix Heatmaps
From the correlation matrix in above heatmap, we see no correlation with the data (extremely low R-values).

#**Baseline Model**

In [ ]:
#Build simple linear regression baseline model

#Filter dataframe to columns that will be used to build regression model
df_slrm = df_flt.drop(columns=['work_year','employment_type','job_title','salary_currency','employee_residence','company_location'])

#Define response variable
Y = df_slrm['salary_in_usd']

#Drop response variable from feature set
X = df_slrm.drop(['salary_in_usd'], axis=1)

#Encode categorical feature and drop one column do to multi-collinearity
dummy_experience = pd.get_dummies(X['experience_level'], prefix='exp', drop_first=True)
dummy_compsize = pd.get_dummies(X['company_size'], prefix='size', drop_first=True)

#Drop columns that are replaced by dummy variables
X.drop(columns=['experience_level','company_size'],inplace=True)

#Assign dummy variables back to the dataframe
X = X.join(dummy_experience)
X = X.join(dummy_compsize)

#Split observations
tX, vX, tY, vY = tts(X, Y, test_size = 0.2, random_state=0)
#Fit the model
mdl = sm.OLS(tY, tX).fit()
#Print OLS Regression summary with significance level of 5%
print(mdl.summary(title='Baseline Model for Full Time Data Scientist Salary in US', alpha=.05))
#Predict salaries using test set 
pY = mdl.predict(vX)
#Compute out-of-sample R^2
print(f'Out of sample R^2 is {r2_score(vY, pY):.2f}')

*   This is pretty poor out-of-sample R2, let's try to add an intercept to the model to see if it improves




In [ ]:
#Add an intercept to the linear regression model to see if we can improve the out-of-sample R2
#Copy dataframe
X2 = X.copy()

#Define response variable
Y2 = Y.copy()

#Split observations
tX2, vX2, tY2, vY2 = tts(X2, Y2, test_size = 0.2, random_state=0)
#Fit the model
mdl2 = sm.OLS(tY2, sm.add_constant(tX2)).fit()
#Print summary stats
print(mdl2.summary(title='Improvement Linear Regression Model for Full Time Data Scientist Salary in US', alpha=.05))
#Predict market value using test set 
pY2 = mdl2.predict(sm.add_constant(vX2))
#Compute out-of-sample R^2
print(f'Out of sample R^2 is {r2_score(vY2, pY2):.2f}')

*   Slightly better out-of-sample R2 but still relatively poor model. 

*   We did not use clustering or logistic regression models as we did not feel it fit our problem statement

*   Let's try Decision Tree Regression










In [ ]:
#Lets try decision tree regressor
#Copy dataframe
X3 = X.copy()

#Define response variable
Y3 = Y.copy()

tX3, vX3, tY3, vY3 = tts(X3, Y3, test_size=0.2, random_state=0)

#Define grid search params
param_grid = [{"max_depth":[3, 4, 5, None], "max_features":[4,5,6]}]

gs = GridSearchCV(estimator=DecisionTreeRegressor(random_state=0),\
                 param_grid = param_grid,\
                 cv=10);

#Fit the model
gs.fit(tX3, tY3)

#Print out best combination or params
print(f'\nBest model params: {gs.best_params_}')

#Print out best estimator 
print(f'Best model estimator: {gs.best_estimator_}')

#Make a prediction 
predictions = gs.predict(vX3)

#Calculate MAE
print(f'MAE : {mean_absolute_error(vY3, predictions)}')

#Calculate R2
print(f'Out of Sample R2: {r2_score(vY3, predictions)}')

Decision Tree Regression model did not improve out-of-sample R2. Next we will try Random Forest model.